# Setting TensorFlow Version and Imports



In [18]:
%tensorflow_version 2.x


In [19]:
from __future__ import absolute_import, division, print_function, unicode_literals
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import zipfile 
import IPython.display as display
from PIL import Image
AUTOTUNE = tf.data.experimental.AUTOTUNE

import cv2



print(tf.version) #confirming version

<module 'tensorflow._api.v2.version' from '/usr/local/lib/python3.7/dist-packages/tensorflow/_api/v2/version/__init__.py'>


In [21]:
print(tf.__version__)

2.4.1


# Importing Kaggle into Google Collab

Need to download a Kaggle.json from the kaggle website

Need to put kaggle API key associated with my account into colab, so create the directory to put it in

In [2]:
!mkdir -p ~/.kaggle

Now, download an API key JSON file from Kaggle under 'My Account' and upload it here:

In [3]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"davidfreiberger","key":"a0b9d5bab6e8d84e17e49323da7b7259"}'}

In [4]:
!cp kaggle.json ~/.kaggle/

# Download the Dataset from Kaggle

In [5]:
!kaggle datasets download -d sriramr/fruits-fresh-and-rotten-for-classification -p /content

100% 3.58G/3.58G [01:06<00:00, 19.8MB/s]
100% 3.58G/3.58G [01:06<00:00, 58.2MB/s]


Opening the saved ZIP file and extracting the contents:

In [6]:
local_zip = '/content/fruits-fresh-and-rotten-for-classification.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/content/fruits-fresh-and-rotten-for-classification')
zip_ref.close()

# Dataset Object Method

## Image Data Generator

In [7]:
# Convenience variables for referencing these directory paths
training_directory = "/content/fruits-fresh-and-rotten-for-classification/dataset/train"
test_directory = "/content/fruits-fresh-and-rotten-for-classification/dataset/dataset/test"

In [8]:
# ImageDataGenerator is a tool in the keras library to automatically pull in images and label them
from tensorflow.keras.preprocessing.image import ImageDataGenerator
image_generator = ImageDataGenerator()

In [9]:
# train_data will contain (image, label) tuples for all training images
train_data = image_generator.flow_from_directory(directory=training_directory,
                                                       shuffle=True,
                                                       target_size=(80,80),
                                                       class_mode='binary')

Found 10901 images belonging to 6 classes.


In [10]:
# same as training, but the testing images
test_data = image_generator.flow_from_directory(directory=test_directory,
                                                target_size=(80,80),
                                                class_mode='binary')

Found 2698 images belonging to 6 classes.


In [11]:
print(train_data.class_indices)
print(test_data.class_indices)

{'freshapples': 0, 'freshbanana': 1, 'freshoranges': 2, 'rottenapples': 3, 'rottenbanana': 4, 'rottenoranges': 5}
{'freshapples': 0, 'freshbanana': 1, 'freshoranges': 2, 'rottenapples': 3, 'rottenbanana': 4, 'rottenoranges': 5}


In [12]:
print(train_data)

# Create The Model

# New Section

In [13]:
# This is just an arbitrary model that will work but can be changed in the future
model = tf.keras.models.Sequential([
                                    # First Convolution
                                    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(80,80,3)),
                                    tf.keras.layers.MaxPooling2D(2,2),
                                    # Second Convolution
                                    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
                                    tf.keras.layers.MaxPooling2D(2,2),
                                    # Third Convolution
                                    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
                                    tf.keras.layers.MaxPooling2D(2,2),
                                    # Flatten the results to feed to a DNN
                                    tf.keras.layers.Flatten(),
                                    tf.keras.layers.Dense(128, activation='relu'),
                                    tf.keras.layers.Dense(64, activation='relu'),
                                    # 6 outputs:
                                    tf.keras.layers.Dense(6, activation='softmax')
])

In [14]:
# Also arbitrary, can be optimized in the future
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [15]:
history = model.fit(train_data, epochs=4)

Epoch 1/4
341/341 [==============================] - 71s 113ms/step - loss: 10.5067 - accuracy: 0.5666
Epoch 2/4
341/341 [==============================] - 39s 115ms/step - loss: 0.3601 - accuracy: 0.8736
Epoch 3/4
341/341 [==============================] - 39s 114ms/step - loss: 0.2574 - accuracy: 0.9070
Epoch 4/4
341/341 [==============================] - 39s 114ms/step - loss: 0.1982 - accuracy: 0.9312


In [16]:
results = model.evaluate(test_data)
print("test loss, test acc:", results)

85/85 [==============================] - 13s 154ms/step - loss: 0.2518 - accuracy: 0.9088
test loss, test acc: [0.25178608298301697, 0.9088213443756104]


In [ ]:
prediction = model.predict(train_data)

# Converting to Tensorflow Lite

In [23]:
# Converting a tf.Keras model to a TensorFlow Lite model
converter = tf.lite.TFLiteConverter.from_keras_model(model)

tflite_model = converter.convert()

INFO:tensorflow:Assets written to: /tmp/tmp1xkicm7n/assets


In [27]:
saved_model_dir = '/content/MLmodel'
!mkdir saved_model_dir

In [25]:
model.save(saved_model_dir)

INFO:tensorflow:Assets written to: /content/MLmodel/assets


INFO:tensorflow:Assets written to: /content/MLmodel/assets


In [28]:
from_saved_converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)
from_saved_tflite_model = from_saved_converter.convert()




In [29]:
# Save the model
with open('model.tflite', 'wb') as f:
  f.write(tflite_model)